In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [2]:
source_dir = '../data/undesired_outcomes_pred/'
dest_dir = source_dir
extension = '.csv'

In [3]:
elog_uo = pd.read_csv(source_dir.__add__('elog_uo.csv'))

In [4]:
pd.options.display.max_columns = 100

In [5]:
elog_uo.sample(10)

,Case ID,Activity,Resource,Complete Timestamp,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,(case) applicant,(case) area,(case) basic payment,(case) cross_compliance,(case) department,(case) greening,(case) number_parcels,(case) payment_actual0,(case) payment_actual1,(case) payment_actual2,(case) payment_actual3,(case) penalty_ABP,(case) penalty_AGP,(case) penalty_AJLP,(case) penalty_AUVP,(case) penalty_AVBP,(case) penalty_AVGP,(case) penalty_AVJLP,(case) penalty_AVUVP,(case) penalty_B16,(case) penalty_B2,(case) penalty_B3,(case) penalty_B4,(case) penalty_B5,(case) penalty_B5F,(case) penalty_B6,(case) penalty_BGK,(case) penalty_BGKV,(case) penalty_BGP,(case) penalty_C16,(case) penalty_C4,(case) penalty_C9,(case) penalty_CC,(case) penalty_GP1,(case) penalty_JLP1,(case) penalty_JLP2,(case) penalty_JLP3,(case) penalty_JLP5,(case) penalty_JLP6,(case) penalty_JLP7,(case) penalty_V5,(case) penalty_amount0,(case) penalty_amount1,(case) penalty_amount2,(case) penalty_amount3,(case) redistribution,(case) rejected,(case) risk_factor,(case) selected_manually,(case) selected_random,(case) selected_risk,(case) small farmer,(case) year,(case) young farmer,activity,docid,doctype,eventid,note,subprocess,success,num changes applied amount,final applied amount,diff applied amount,applied amount lowered,applied amount changed,num changes actual amount,final actual amount,diff actual amount,actual amount lowered,actual amount changed,actual amount pay,num changes penalty amount,final penalty amount,penalty amount applied,num_penalty_reasons,num_severe_penalties,penalty applied,severe penalty applied,timestamp_year,timestamp_month,undesired_outcome1,undesired_outcome2,undesired_outcome
31601,a937191fe79fdc4b,Payment application-Application-finish editing,727350,2017-11-17 11:34:59.098,1504.00,NaN,NaN,NaN,6f11751c1ce665de,4.3908,True,0.0,e7,True,2,1504.00,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2017,False,finish editing,ff40057d15c118d7,Payment application,ffc0057f1b2e9982,automatic,Application,True,0,1504.00,0.0,False,False,0,1504.00,0.0,False,False,False,0,0.0,False,0,0,False,False,2017,11,True,False,True
43395,7bf477c700000941,Payment application-Application-finish editing,727350,2017-11-17 08:36:23.886,10902.58,NaN,NaN,NaN,b9a3889342d85787,32.6797,True,0.0,6b,True,12,10899.68,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2017,False,finish editing,ff40057d15ad8e2b,Payment application,ffc0057f1a6961fc,automatic,Application,True,0,10902.58,0.0,False,False,0,10899.68,0.0,False,False,False,0,0.0,False,0,0,False,False,2017,11,True,False,True
38605,1968b991620a98ea,Payment application-Application-finish editing,727350,2017-11-17 11:03:52.150,30526.40,NaN,NaN,NaN,c13204470bd97bea,89.8664,True,0.0,4e,True,28,30517.06,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2017,True,finish editing,ff40057d15bac723,Payment application,ffc0057f1b11b1fc,automatic,Application,True,0,30526.40,0.0,False,False,0,30517.06,0.0,False,False,False,0,0.0,False,0,0,False,False,2017,11,True,False,True
36472,7a336fc3f2388617,Payment application-Application-finish editing,727350,2017-11-17 07:07:01.722,4476.84,NaN,NaN,NaN,237be8612dd438b8,13.3202,True,0.0,6b,True,4,4476.08,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [6]:
target = np.array(elog_uo['undesired_outcome'])

In [7]:
elog_uo.columns

Index(['Case ID', 'Activity', 'Resource', 'Complete Timestamp',
       '(case) amount_applied0', '(case) amount_applied1',
       '(case) amount_applied2', '(case) amount_applied3', '(case) applicant',
       '(case) area', '(case) basic payment', '(case) cross_compliance',
       '(case) department', '(case) greening', '(case) number_parcels',
       '(case) payment_actual0', '(case) payment_actual1',
       '(case) payment_actual2', '(case) payment_actual3',
       '(case) penalty_ABP', '(case) penalty_AGP', '(case) penalty_AJLP',
       '(case) penalty_AUVP', '(case) penalty_AVBP', '(case) penalty_AVGP',
       '(case) penalty_AVJLP', '(case) penalty_AVUVP', '(case) penalty_B16',
       '(case) penalty_B2', '(case) penalty_B3', '(case) penalty_B4',
       '(case) penalty_B5', '(case) penalty_B5F', '(case) penalty_B6',
       '(case) penalty_BGK', '(case) penalty_BGKV', '(case) penalty_BGP',
       '(case) penalty_C16', '(case) penalty_C4', '(case) penalty_C9',
       '(case) penalty

In [8]:
features = ['(case) amount_applied0', '(case) area', '(case) department', '(case) number_parcels', '(case) payment_actual0',
           '(case) penalty_amount0', '(case) penalty_B2', '(case) year', 'final applied amount',
             'final actual amount', 'final penalty amount', 'num_penalty_reasons', 'num_severe_penalties',
            'num changes actual amount', 'num changes applied amount', 'num changes penalty amount', 'penalty amount applied', 
            'penalty applied', 'severe penalty applied', 'timestamp_month', 'timestamp_year']

In [9]:
elog_features = elog_uo[features]

In [10]:
elog_features.shape[1] == len(features)

True

In [11]:
elog_features = pd.get_dummies(elog_features) # one hot encoding

In [12]:
elog_features_np = np.array(elog_features)
elog_features_col = list(elog_features.columns)

In [13]:
train_x, test_x, train_y, test_y = train_test_split(elog_features_np, target, test_size = 0.3, random_state = 99)

In [14]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 99)

In [15]:
rf.fit(train_x, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=99,
           verbose=0, warm_start=False)

In [16]:
pred = rf.predict(test_x)
pred = pred.astype(int)

In [17]:
errors = abs(pred - test_y)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.04 degrees.


In [18]:
mape = 100 * (errors / test_y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [19]:
accuracy = accuracy_score(test_y, pred)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 0.96 %.


In [20]:
confusion_matrix(test_y, pred)

array([[6694,    0],
       [ 588, 5856]])